In [1]:
import pandas as pd 
import numpy as np
import lightgbm as lgb


In [3]:
df = pd.read_pickle("../data/processed/df_pp.pickle")

In [4]:
df.head()

,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time,pid,req_time,o_long,o_lat,d_long,d_lat,distance_query
0,2848914,2018-11-17 18:42:17,1,53156,6456,700.0,1,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
1,2848914,2018-11-17 18:42:17,1,48112,3535,700.0,3,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
2,2848914,2018-11-17 18:42:17,1,48112,3655,16500.0,4,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
3,2848914,2018-11-17 18:42:17,1,51641,8871,1200.0,1,2018-11-17 12:56:15,101804.0,2018-11-17 12:56:15,116.36,40.07,116.00,40.35,43.656570
4,2629085,2018-10-12 16:28:13,3,13207,2790,400.0,9,2018-10-12 16:27:55,203797.0,2018-10-12 16:27:55,116.35,40.08,116.33,40.03,5.808139


In [5]:
def test_train_split_df(df_r, test_percentage=0.25):
    import numpy as np
    df = df_r.copy()
    
    msk = np.random.rand(len(df)) < (1-test_percentage)
    
    train = df[msk]
    test = df[~msk]
    
    
    return train, test
    

In [6]:
df_X = df[["sid", "distance_plan", "eta", "price", "distance_query", "click_mode"]]

In [7]:
train, test = test_train_split_df(df_X)

In [8]:
x_train = train.drop(["sid", "click_mode"], axis=1).values
y_train = train.loc[:,"click_mode"].values
x_test = test.drop(["sid", "click_mode"], axis=1).values
y_test = test.loc[:,"click_mode"].values

In [23]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1576375, 4)
(1576375,)
(524685, 4)
(524685,)


In [71]:
def lgb_f1_score(y_true, y_pred):
    from sklearn.metrics import f1_score
    preds = y_pred.reshape(len(np.unique(y_true)), -1)
    preds = preds.argmax(axis = 0)
    #print(preds.shape)
    #print(y_true.shape)
    return 'f1', f1_score(y_true, preds,average='weighted'), True

In [73]:

lgb_model = lgb.LGBMClassifier(boosting_type="gbdt",
                               num_leaves=61, reg_alpha=0,
                               reg_lambda=0.01,max_depth=-1,
                               n_estimators=2000, objective='multiclass',
                               subsample=0.8, colsample_bytree=0.8, 
                               subsample_freq=1,min_child_samples = 50,  
                               learning_rate=0.05, random_state=2019, metric="None",n_jobs=-1)


eval_set = [(x_test, y_test)]


lgb_model.fit(x_train, y_train, 
              eval_set=eval_set, 
              eval_metric=lgb_f1_score, 
              verbose=10, 
              early_stopping_rounds=100)



/home/sandro/anaconda3/envs/lbm_mc_bl/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's f1: 0.357725
[20]	valid_0's f1: 0.359514
[30]	valid_0's f1: 0.360389
[40]	valid_0's f1: 0.361561
[50]	valid_0's f1: 0.36237
[60]	valid_0's f1: 0.36301
[70]	valid_0's f1: 0.363822
[80]	valid_0's f1: 0.364456
[90]	valid_0's f1: 0.365112
[100]	valid_0's f1: 0.365577
[110]	valid_0's f1: 0.366026
[120]	valid_0's f1: 0.366479
[130]	valid_0's f1: 0.366927
[140]	valid_0's f1: 0.367217
[150]	valid_0's f1: 0.367518
[160]	valid_0's f1: 0.367714
[170]	valid_0's f1: 0.367942
[180]	valid_0's f1: 0.368101
[190]	valid_0's f1: 0.368187
[200]	valid_0's f1: 0.368428
[210]	valid_0's f1: 0.368579
[220]	valid_0's f1: 0.36873
[230]	valid_0's f1: 0.3688
[240]	valid_0's f1: 0.369031
[250]	valid_0's f1: 0.369221
[260]	valid_0's f1: 0.369361
[270]	valid_0's f1: 0.369535
[280]	valid_0's f1: 0.36976
[290]	valid_0's f1: 0.369863
[300]	valid_0's f1: 0.370021
[310]	valid_0's f1: 0.370132
[320]	valid_0's f1: 0.370222
[330]	valid_0's f1: 0.37

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        importance_type='split', learning_rate=0.05, max_depth=-1,
        metric='None', min_child_samples=50, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=61,
        objective='multiclass', random_state=2019, reg_alpha=0,
        reg_lambda=0.01, silent=True, subsample=0.8,
        subsample_for_bin=200000, subsample_freq=1)

In [75]:
import pickle
import os
filename = os.path.join("..", "models", "lgbmodel_2k.pickle")
pickle.dump(lgb_model, open(filename, 'wb'))